In [1]:
import numpy as np
import pandas as pd
import os
from anomaly_detection.traj_combiner import TrajCombiner
from anomaly_detection.OD import OD
from toll_detection.road_network import RoadNetwork
from toll_detection.node_set import NodeSet
from toll_detection.poi_tolls import POITolls

%load_ext autoreload
%autoreload 2

### 载入数据&初始化

In [2]:
import json
with open('config.json', 'rb') as f:
    config = json.load(f)
traj_pkl_path = config['combined_traj_withspeed_pkl']
tb = TrajCombiner(pkl_path=traj_pkl_path)
tb.df_combined.head()

,oid,s_time,e_time,coords,timestamps,start_pos,end_pos,length,start_warehoue,end_warehoue,start_city,end_city,origin_idx,warehouse_od,city_od,speeds
0,粤AAM451,1585678875,1585698021,"[[23.15363, 113.450335], [23.154158, 113.45053...","[1585678875, 1585678882, 1585678890, 158567889...","[23.15363, 113.450335]","[23.512176, 116.433503]",400.863902,Guangzhou2,JieYang0,Guangzhou,JieYang,[1547],Guangzhou2:JieYang0,Guangzhou:JieYang,"[8.827107020016035, 8.866938631391324, 10.6106..."
1,粤AAM451,1585758779,1585778111,"[[23.153721, 113.450361], [23.154376, 113.4505...","[1585758779, 1585758785, 1585758789, 158575879...","[23.153721, 113.450361]","[23.511591, 116.431907]",401.023132,Guangzhou2,JieYang0,Guangzhou,JieYang,[927],Guangzhou2:JieYang0,Guangzhou:JieYang,"[12.659468677070272, 12.040510645764403, 8.766..."
2,粤AAM451,1585787410,1585807270,"[[23.524477, 116.436018], [23.524432, 116.4360...","[1585787410, 1585787412, 1585787418, 158578742...","[23.524477, 116.436018]","[23.072217, 113.592191]",399.347198,JieYang0,DongGuan0,JieYang,DongGuan,[311],JieYang0:DongGuan0,JieYang:DongGuan,"[2.500262817903009, 2.8073135084707688, 4.0602..."
3,粤AAM451,1585849312,1585895375,"[[23.155013, 113.450775], [23.155585, 113.4509...","[1585849312, 1585849320, 1585849324, 158584932...","[23.155013, 113.450775]","[22.754265, 108.283778]",668.619147,Guangzhou2,NanNing0,Guangzhou,NanNing,[2049],Guangzhou2:NanNing0,Guangzhou:NanNing,"[8.195198040544264, 1.8401417280681405, 0.1877..."
4,粤AAM451,1585901261,1585936335,"[[22.755703, 108.287423], [22.75573, 108.28742...","[1585901261, 1585901263, 1585901268, 158590127...","[22.755703, 108.287423]","[23.153323, 113.450166]",661.086279,NanNing0,Guangzhou2,NanNing,Guangzhou,"[2050, 2384]",NanNing0:Guangzhou2,NanNing:Guangzhou,"[1.4949982239325605, 1.1738504630795, 0.375728..."


In [3]:
# 初始化toll和Road
poi_toll_path = config['poi_toll_path']
rn_path = config['roadnetwork_path']
rn = RoadNetwork(rn_path, check_node_consistency=False, parse_full_road=True)
intersection_set = rn.get_highway_intersections_coords()

In [4]:
proper_intersection_set = intersection_set.get_subset_with_specified_type(['primary','secondary', 'tertiary', 'trunk'])
motorway_nodes = rn.get_highway_nodes(lon_first=False)
tolls = POITolls(poi_toll_path, proper_intersection_set, motorway_nodes, build_toll_tree=True)

Number of filtered coords at first round 260
Number of readded coords: 45


## 茂名到广州段

In [5]:
od_mg = tb.get_OD_pairs('MaoMing', 'Guangzhou')

In [6]:
# 检测经过的收费站
od_mg.init_detector(tolls)
od_mg.detect_toll()

In [7]:
od_mg.df.groupby(['start_warehoue', 'end_warehoue']).size()

start_warehoue  end_warehoue
MaoMing0        Guangzhou0        6
                Guangzhou1      119
                Guangzhou2       71
MaoMing1        Guangzhou0       63
                Guangzhou1       18
dtype: int64

In [8]:
od_mg.df.groupby(['start_warehoue', 'uptoll_class']).agg({'uptoll_dis':['mean', 'size']})

uptoll_dis     
                                     mean size
start_warehoue uptoll_class                   
MaoMing0       57            14245.109816    1
               310           11049.479745    3
               402           11681.681642   57
               438           20404.280965   26
               507           15473.447960   63
               569           19300.657713   46
MaoMing1       154           27905.262310    1
               310           24794.235920    1
               402            7811.001788   27
               438           18063.946509   14
               507            9908.223351   28
               569           13411.479427   10

In [9]:
od_mg.vis_uptoll_ratio('Guangzhou2', start=False)

997


## 单独测试TrajTollDetector

In [10]:
from anomaly_detection.detect_toll_of_traj import TrajTollDetector

In [11]:
toll_detector = TrajTollDetector(tolls)

In [12]:
# 茂名段异常轨迹
test_id = 213
toll_detector.vis_traj_with_topk_toll(od_mg.df.loc[test_id,'coords'], od_mg.df.loc[test_id,'speeds'])